In [1]:
from catboost import CatBoostRegressor
import catboost
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ipynb.fs.full.utils as utils
import ipynb.fs.full.features as features
import ipynb.fs.full.features_distancias as f_distancias
import hyperopt
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials

df_train = pd.read_csv('./data/train_filtrado.csv')
# Para usarse con el submit a Kaggle
df_eval = pd.read_csv('./data/test.csv')

df_train, df_eval = features.features_de_csvs(df_train, df_eval)

df_train_idf = pd.read_csv('./data/train_idf.csv')
df_eval_idf = pd.read_csv('./data/test_idf.csv')

df_train = pd.merge(df_train, df_train_idf, on= 'id', how= 'left')
df_eval = pd.merge(df_eval, df_eval_idf, on= 'id', how= 'left')

df_train, df_test = utils.dividir_df_testeo(df_train, test_size=0.01)

df_test = features.llenar_nulls(df_test, hgb_mean=True, df_fill=df_train)
df_train = features.llenar_nulls(df_train, hgb_mean=True)

In [2]:
df_test_f = features.features_independientes_precio(df_test)
df_test_f = features.features_dependientes_precio(df_test_f, df_train)

df_train_f = features.features_independientes_precio(df_train)
df_train_f = features.features_dependientes_precio(df_train_f, df_train)

df_test_f, cols_tipodepropiedad_ohe = features.columna_a_ohe(df_test_f, 'tipodepropiedad', N=100, df_aux=df_train, devolver_cols=True)
df_test_f, cols_provincia_ohe = features.columna_a_ohe(df_test_f, 'provincia', N=100, df_aux=df_train, devolver_cols=True)
df_test_f, cols_zona_ohe = features.columna_a_ohe(df_test_f, 'zona', df_aux=df_train_f, devolver_cols=True)

df_train_f = features.columna_a_ohe(df_train_f, 'tipodepropiedad', N=100, df_aux=df_test)
df_train_f = features.columna_a_ohe(df_train_f, 'provincia', N=100, df_aux=df_test)
df_train_f = features.columna_a_ohe(df_train_f, 'zona', df_aux=df_test_f)


df_train_f['fecha'] = pd.to_datetime(df_train_f['fecha']).astype(int)
df_test_f['fecha'] = pd.to_datetime(df_test_f['fecha']).astype(int)

df_train_f = f_distancias.feature_distancias(df_train_f)
df_test_f = f_distancias.feature_distancias(df_test_f, df_train_f)

In [3]:
# df_train_f = features.features_intervalos(df_train_f)
# df_test_f = features.features_intervalos(df_test_f)

In [ ]:
# # Paso a integer las features

# def feature_convertion(df,features, multiplicar):
#     df = df.copy()
    
#     for feature in features:
#         if multiplicar:
#             df[feature] = df[feature].map(lambda x: x*1000)
#         df[feature] = df[feature].astype('int64')
        
#     return df


# features_int = ['ditancia_centro_bins_unif','ditancia_centro_bins_perc','distancia_centro_mexico_bins_unif',
#             'distancia_centro_mexico_bins_perc','distancia_ciudad_cara_bins_unif',
#             'distancia_ciudad_cara_bins_perc','promedio_id_zona_bins_unif', 'promedio_id_zona_bins_perc',
#             'promedio_precio_hbg_tipo_propiedad_provincia_bins_unif',
#             'promedio_precio_hbg_tipo_propiedad_provincia_bins_perc', 
#             'promedio_precio_tipo_propiedad_ciudad_bins_unif','promedio_precio_tipo_propiedad_ciudad_bins_perc',
#             'antiguedad_bins_unif', 'antiguedad_bins_perc', 'promedio_precio_hbg_tipo_propiedad_provincia_bins_perc',
#             'promedio_precio_hbg_tipo_propiedad_provincia_bins_unif', 'tam_ambientes_bins_perc',
#             'promedio_precio_hbg_tipo_propiedad_provincia_bins_unif', 'habitaciones', 'banos',
#             'metroscubiertos', 'metrostotales', 'count_id_zona', 'antiguedad','garages', 'gimnasio',
#             'usosmultiples', 'piscina', 'gimnasio']

# df_train_f = feature_convertion(df_train_f, features_int, False)

In [ ]:
# features_mult = ['antiguedad', 'habitaciones', 'garages', 'banos', 'metroscubiertos',
#        'metrostotales', 'lat', 'lng', 'fecha', 'piscina', 'escuelascercanas',
#        'centroscomercialescercanos', 'clustering_label', 'idf_titulo',
#        'idf_descripcion', 'peso_titulo', 'peso_descripcion',
#        'porcentaje_metros', 'diferencia_metros', 'metros_totales_normalizados',
#        'metros_cubiertos_normalizados', 'escomercial',
#        'promedio_metros_tipo_propiedad', 'promedio_metros_cub_tipo_propiedad',
#        'prop_frecuente', 'top_provincia', 'es_ciudad_centrica',
#        'promedio_metros_totales_provincia',
#        'promedio_metros_cubiertos_provincia', 'mes', 'dia', 'trimestre',
#        'dias_desde_datos', 'meses_desde_datos', 'antiguedad_bins_perc',
#        'cantidad_inquilinos', 'tam_ambientes', 'promedio_precio_provincia',
#        'promedio_precio_ciudad', 'varianza_precio_ciudad', 'count_ciudad',
#        'promedio_id_zona', 'promedio_id_zona_gen', 'varianza_id_zona',
#        'count_id_zona', 'promedio_precio_tipo_propiedad',
#        'promedio_precio_tipo_propiedad_ciudad',
#        'promedio_precio_tipo_propiedad_ciudad_gen', 'count_tipo_propiedad',
#        'count_tipo_propiedad_ciudad', 'promedio_por_mes', 'varianza_por_mes',
#        'promedio_precio_habitaciones',
#        'promedio_precio_habitaciones_banos_garages',
#        'promedio_precio_banos_garages', 'promedio_precio_hbg_tipo_propiedad',
#        'lat_norm', 'lng_norm', 'promedio_precio_booleanos', 'puntaje',
#        'distancia_ciudad_centrica', 'distancia_centro_mexico',
#        'distancia_ciudad_cara']


# df_train_f = feature_convertion(df_train_f, features_mult, True)

# df_train_f['lat'] = df_train_f['lat'].map(lambda x: x*1000000)
# df_train_f['lng'] = df_train_f['lng'].map(lambda x: x*1000000)
# df_train_f['lat'] = df_train_f['lng'].astype('int64')
# df_train_f['lng'] = df_train_f['lng'].astype('int64')

## Busqueda de Hiperparametros

### GridSearchCV

In [5]:
from sklearn.model_selection import GridSearchCV


features = ['antiguedad', 'habitaciones', 'garages', 'banos', 'metroscubiertos',
       'metrostotales', 'lat', 'lng', 'fecha', 'piscina', 'escuelascercanas',
       'centroscomercialescercanos', 'clustering_label', 'idf_titulo',
       'idf_descripcion', 'peso_titulo', 'peso_descripcion',
       'porcentaje_metros', 'diferencia_metros', 'metros_totales_normalizados',
       'metros_cubiertos_normalizados', 'escomercial',
       'promedio_metros_tipo_propiedad', 'promedio_metros_cub_tipo_propiedad',
       'prop_frecuente', 'top_provincia', 'es_ciudad_centrica',
       'promedio_metros_totales_provincia',
       'promedio_metros_cubiertos_provincia', 'mes', 'dia', 'trimestre',
       'dias_desde_datos', 'meses_desde_datos',
       'cantidad_inquilinos', 'tam_ambientes', 'promedio_precio_provincia',
       'promedio_precio_ciudad', 'varianza_precio_ciudad', 'count_ciudad',
       'promedio_id_zona', 'promedio_id_zona_gen', 'varianza_id_zona',
       'count_id_zona', 'promedio_precio_tipo_propiedad',
       'promedio_precio_tipo_propiedad_ciudad',
       'promedio_precio_tipo_propiedad_ciudad_gen', 'count_tipo_propiedad',
       'count_tipo_propiedad_ciudad', 'promedio_por_mes', 'varianza_por_mes',
       'promedio_precio_habitaciones',
       'promedio_precio_habitaciones_banos_garages',
       'promedio_precio_banos_garages', 'promedio_precio_hbg_tipo_propiedad',
       'lat_norm', 'lng_norm', 'promedio_precio_booleanos', 'puntaje',
       'distancia_ciudad_centrica', 'distancia_centro_mexico',
       'distancia_ciudad_cara']

cat_params = {
    'learning_rate': [0.1, 0.15 , 0.2],
    'depth': [6, 8, 10, ],
    'od_type': ['Iter'],
    'od_wait': [10],
    'border_count': [64, 128, 176, 254, 512],
    'l2_leaf_reg': [0, 5, 10, 15, 35, 50, 75, 100, 200]
}


x_train, x_test, y_train, y_test = utils.dividir_dataset(df_train_f, 'precio', features, test_size=1)

gs_cv = GridSearchCV(estimator=CatBoostRegressor(iterations=50, eval_metric='MAE', silent=True, ),
                           param_grid=cat_params, 
                           cv=3, 
                           verbose=3, scoring='neg_mean_absolute_error')

gs_cv.fit(x_train, y_train, verbose=False)

Fitting 3 folds for each of 270 candidates, totalling 810 fits
[CV] border_count=64, depth=6, l2_leaf_reg=0, learning_rate=0.1, od_type=Iter, od_wait=10 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  border_count=64, depth=6, l2_leaf_reg=0, learning_rate=0.1, od_type=Iter, od_wait=10, score=-586874.956, total=   2.3s
[CV] border_count=64, depth=6, l2_leaf_reg=0, learning_rate=0.1, od_type=Iter, od_wait=10 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.3s remaining:    0.0s


[CV]  border_count=64, depth=6, l2_leaf_reg=0, learning_rate=0.1, od_type=Iter, od_wait=10, score=-585933.158, total=   2.1s
[CV] border_count=64, depth=6, l2_leaf_reg=0, learning_rate=0.1, od_type=Iter, od_wait=10 


...

[CV]  border_count=512, depth=8, l2_leaf_reg=15, learning_rate=0.1, od_type=Iter, od_wait=10, score=-566490.531, total=  10.3s
[CV] border_count=512, depth=8, l2_leaf_reg=15, learning_rate=0.2, od_type=Iter, od_wait=10 
[CV]  border_count=512, depth=8, l2_leaf_reg=15, learning_rate=0.2, od_type=Iter, od_wait=10, score=-540011.829, total=   7.4s
[CV] border_count=512, depth=8, l2_leaf_reg=15, learning_rate=0.2, od_type=Iter, od_wait=10 
[CV]  border_count=512, depth=8, l2_leaf_reg=15, learning_rate=0.2, od_type=Iter, od_wait=10, score=-540928.027, total=   5.9s
[CV] border_count=512, depth=8, l2_leaf_reg=15, learning_rate=0.2, od_type=Iter, od_wait=10 
[CV]  border_count=512, depth=8, l2_leaf_reg=15, learning_rate=0.2, od_type=Iter, od_wait=10, score=-540410.883, total=   7.0s
[CV] border_count=512, depth=8, l2_leaf_reg=35, learning_rate=0.1, od_type=Iter, od_wait=10 
[CV]  border_count=512, depth=8, l2_leaf_reg=35, learning_rate=0.1, od_type=Iter, od_wait=10, score=-569616.527, total= 

[CV]  border_count=512, depth=10, l2_leaf_reg=0, learning_rate=0.2, od_type=Iter, od_wait=10, score=-521253.005, total=  12.2s
[CV] border_count=512, depth=10, l2_leaf_reg=0, learning_rate=0.2, od_type=Iter, od_wait=10 
[CV]  border_count=512, depth=10, l2_leaf_reg=0, learning_rate=0.2, od_type=Iter, od_wait=10, score=-520328.515, total=  12.8s
[CV] border_count=512, depth=10, l2_leaf_reg=5, learning_rate=0.1, od_type=Iter, od_wait=10 
[CV]  border_count=512, depth=10, l2_leaf_reg=5, learning_rate=0.1, od_type=Iter, od_wait=10, score=-547549.261, total=  12.8s
[CV] border_count=512, depth=10, l2_leaf_reg=5, learning_rate=0.1, od_type=Iter, od_wait=10 
[CV]  border_count=512, depth=10, l2_leaf_reg=5, learning_rate=0.1, od_type=Iter, od_wait=10, score=-547759.686, total=  15.6s
[CV] border_count=512, depth=10, l2_leaf_reg=5, learning_rate=0.1, od_type=Iter, od_wait=10 
[CV]  border_count=512, depth=10, l2_leaf_reg=5, learning_rate=0.1, od_type=Iter, od_wait=10, score=-546353.696, total= 

[CV]  border_count=512, depth=10, l2_leaf_reg=75, learning_rate=0.2, od_type=Iter, od_wait=10, score=-545735.468, total=  12.0s
[CV] border_count=512, depth=10, l2_leaf_reg=100, learning_rate=0.1, od_type=Iter, od_wait=10 
[CV]  border_count=512, depth=10, l2_leaf_reg=100, learning_rate=0.1, od_type=Iter, od_wait=10, score=-565243.883, total=  12.4s
[CV] border_count=512, depth=10, l2_leaf_reg=100, learning_rate=0.1, od_type=Iter, od_wait=10 
[CV]  border_count=512, depth=10, l2_leaf_reg=100, learning_rate=0.1, od_type=Iter, od_wait=10, score=-566508.636, total=  12.5s
[CV] border_count=512, depth=10, l2_leaf_reg=100, learning_rate=0.1, od_type=Iter, od_wait=10 
[CV]  border_count=512, depth=10, l2_leaf_reg=100, learning_rate=0.1, od_type=Iter, od_wait=10, score=-564972.626, total=  12.3s
[CV] border_count=512, depth=10, l2_leaf_reg=100, learning_rate=0.2, od_type=Iter, od_wait=10 
[CV]  border_count=512, depth=10, l2_leaf_reg=100, learning_rate=0.2, od_type=Iter, od_wait=10, score=-54

[Parallel(n_jobs=1)]: Done 810 out of 810 | elapsed: 79.2min finished


GridSearchCV(cv=3, error_score='raise-deprecating',
             estimator=<catboost.core.CatBoostRegressor object at 0x7f24649bc198>,
             iid='warn', n_jobs=None,
             param_grid={'border_count': [64, 128, 176, 254, 512],
                         'depth': [6, 8, 10],
                         'l2_leaf_reg': [0, 5, 10, 15, 35, 50, 75, 100, 200],
                         'learning_rate': [0.1, 0.2], 'od_type': ['Iter'],
                         'od_wait': [10]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='neg_mean_absolute_error', verbose=3)

In [6]:
display(gs_cv.best_params_)
from sklearn.externals import joblib
# Guarda el modelo
joblib.dump(gs_cv, 'catboost_gscv_1.pkl', compress=1)

{'border_count': 176,
 'depth': 10,
 'l2_leaf_reg': 0,
 'learning_rate': 0.2,
 'od_type': 'Iter',
 'od_wait': 10}

/home/camix/anaconda3/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


['catboost_gscv_1.pkl']

In [7]:
pd.DataFrame(gs_cv.cv_results_).sort_values('rank_test_score')

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_border_count,param_depth,param_l2_leaf_reg,param_learning_rate,param_od_type,param_od_wait,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
145,10.381730,0.182219,0.137621,0.003483,176,10,0,0.2,Iter,10,"{'border_count': 176, 'depth': 10, 'l2_leaf_re...",-519947.801817,-520754.975116,-518865.297344,-519856.025151,774.180859,1
37,7.946428,0.718248,0.090777,0.002163,64,10,0,0.2,Iter,10,"{'border_count': 64, 'depth': 10, 'l2_leaf_reg...",-520913.320565,-521685.925327,-518407.920894,-520335.724731,1399.174209,2
91,7.812444,0.071799,0.085449,0.003442,128,10,0,0.2,Iter,10,"{'border_count': 128, 'depth': 10, 'l2_leaf_re...",-520602.667488,-521320.317393,-519549.716725,-520490.901013,727.150723,3
253,12.444941,0.231465,0.094787,0.013885,512,10,0,0.2,Iter,10,"{'border_count': 512, 'depth': 10, 'l2_leaf_re...",-520782.667046,-521253.005164,-520328.515302,-520788.062481,377.439882,4
39,6.812810,0.259150,0.090105,0.004800,64,10,5,0.2,Iter,10,"{'border_count': 64, 'depth': 10, 'l2_leaf_reg...",-523137.547497,-519763.708741,-520173.587433,-521024.956921,1503.179317,5
199,12.702331,0.755541,0.112953,0.010631,254,10,0,0.2,Iter,10,"{'border_count': 254, 'depth': 10, 'l2_leaf_re...",-521254.636099,-521362.153698,-520892.760495,-521169.850460,200.788084,6
93,7.577267,0.220956,0.079683,0.000382,128,10,5,0.2,Iter,10,"{'border_count': 128, 'depth': 10, 'l2_leaf_re...",-522199.373511,-521774.690733,-522009.839447,-521994.635439,173.709205,7
41,6.808074,0.065810,0.090401,0.010168,64,10,10,0.2,Iter,10,"{'border_count': 64, 'depth': 10, 'l2_leaf_reg...",-522455.820512,-523400.422585,-522083.974880,-522646.738510,554.132595,8
255,16.419377,0.199335,0.120693,0.023880,512,10,5,0.2,Iter,10,"{'border_count': 512, 'depth': 10, 'l2_leaf_re...",-524094.219852,-523063.709575,-522628.531599,-523262.157232,614.598827,9
201,13.577251,0.822354,0.118951,0.019619,254,10,5,0.2,Iter,10,"{'border_count': 254, 'depth': 10, 'l2_leaf_re...",-523330.401992,-522863.627421,-524780.003645,-523658.009619,815.931629,10


### RandomizedSearchCV

In [17]:
from sklearn.model_selection import RandomizedSearchCV


features = ['antiguedad', 'habitaciones', 'garages', 'banos', 'metroscubiertos',
       'metrostotales', 'lat', 'lng', 'fecha', 'piscina', 'escuelascercanas',
       'centroscomercialescercanos', 'clustering_label', 'idf_titulo',
       'idf_descripcion', 'peso_titulo', 'peso_descripcion',
       'porcentaje_metros', 'diferencia_metros', 'metros_totales_normalizados',
       'metros_cubiertos_normalizados', 'escomercial',
       'promedio_metros_tipo_propiedad', 'promedio_metros_cub_tipo_propiedad',
       'prop_frecuente', 'top_provincia', 'es_ciudad_centrica',
       'promedio_metros_totales_provincia',
       'promedio_metros_cubiertos_provincia', 'mes', 'dia', 'trimestre',
       'dias_desde_datos', 'meses_desde_datos',
       'cantidad_inquilinos', 'tam_ambientes', 'promedio_precio_provincia',
       'promedio_precio_ciudad', 'varianza_precio_ciudad', 'count_ciudad',
       'promedio_id_zona', 'promedio_id_zona_gen', 'varianza_id_zona',
       'count_id_zona', 'promedio_precio_tipo_propiedad',
       'promedio_precio_tipo_propiedad_ciudad',
       'promedio_precio_tipo_propiedad_ciudad_gen', 'count_tipo_propiedad',
       'count_tipo_propiedad_ciudad', 'promedio_por_mes', 'varianza_por_mes',
       'promedio_precio_habitaciones',
       'promedio_precio_habitaciones_banos_garages',
       'promedio_precio_banos_garages', 'promedio_precio_hbg_tipo_propiedad',
       'lat_norm', 'lng_norm', 'promedio_precio_booleanos', 'puntaje',
       'distancia_ciudad_centrica', 'distancia_centro_mexico',
       'distancia_ciudad_cara']

cat_params = {
    'learning_rate': [0.05, 0.1, 0.15, 0.2],
    'depth': [6, 8, 10, 12],
#     'bagging_temperature': [0.2],
    'od_type': ['Iter'],
#     'metric_period': [1],   # No lo encuentro, no se de donde salio
    'od_wait': [10, 15, 30, 50, 80, 100],
    'l2_leaf_reg': [0, 1,2 ,3, 5,8, 10, 15,30, 50],
    'border_count': [64, 128, 150 , 176, 254],
    'colsample_bylevel': [0.01, 0.1, 0.3, 0.5, 0.7, 1]
    
}


x_train, x_test, y_train, y_test = utils.dividir_dataset(df_train_f, 'precio', features, test_size=1)

rs_cv = RandomizedSearchCV(estimator=CatBoostRegressor(iterations=150, eval_metric='MAE', silent=True,),
                           param_distributions=cat_params, 
                           cv=3, 
                           n_iter=50,
                           verbose=3, scoring='neg_mean_absolute_error')

rs_cv.fit(x_train, y_train, verbose=1)

Fitting 3 folds for each of 50 candidates, totalling 150 fits
[CV] od_wait=30, od_type=Iter, learning_rate=0.2, l2_leaf_reg=2, depth=8, colsample_bylevel=0.01, border_count=128 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


0:	learn: 1580570.5266606	total: 8.4ms	remaining: 1.25s
1:	learn: 1580570.5224613	total: 15.5ms	remaining: 1.15s
2:	learn: 1521363.3778618	total: 23.8ms	remaining: 1.17s
3:	learn: 1510176.3156855	total: 30.8ms	remaining: 1.13s
4:	learn: 1510176.3609125	total: 37.6ms	remaining: 1.09s
5:	learn: 1510176.3970940	total: 43.7ms	remaining: 1.05s
6:	learn: 1500168.9993208	total: 52.6ms	remaining: 1.07s
7:	learn: 1500169.0571530	total: 58.6ms	remaining: 1.04s
8:	learn: 1463345.0232426	total: 69.5ms	remaining: 1.09s
9:	learn: 1463345.0782288	total: 77.3ms	remaining: 1.08s
10:	learn: 1351450.0741127	total: 85.5ms	remaining: 1.08s
11:	learn: 1351450.2601413	total: 93.6ms	remaining: 1.08s
12:	learn: 1351450.4089649	total: 99.6ms	remaining: 1.05s
13:	learn: 1309188.1292202	total: 112ms	remaining: 1.09s
14:	learn: 1309191.6919231	total: 118ms	remaining: 1.06s
15:	learn: 1302695.8979046	total: 125ms	remaining: 1.05s
16:	learn: 1302698.2104742	total: 132ms	remaining: 1.03s
17:	learn: 1293349.1489346	to

147:	learn: 788759.6366105	total: 1.22s	remaining: 16.4ms
148:	learn: 787823.2358554	total: 1.23s	remaining: 8.26ms
149:	learn: 787104.3136741	total: 1.24s	remaining: 0us
[CV]  od_wait=30, od_type=Iter, learning_rate=0.2, l2_leaf_reg=2, depth=8, colsample_bylevel=0.01, border_count=128, score=-785621.018, total=   2.0s
[CV] od_wait=30, od_type=Iter, learning_rate=0.2, l2_leaf_reg=2, depth=8, colsample_bylevel=0.01, border_count=128 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.0s remaining:    0.0s


...

68:	learn: 1046273.9504552	total: 621ms	remaining: 729ms
69:	learn: 1040432.3557522	total: 631ms	remaining: 721ms
70:	learn: 1040432.4505567	total: 639ms	remaining: 711ms
71:	learn: 1039852.2568430	total: 650ms	remaining: 704ms
72:	learn: 1036246.4604693	total: 659ms	remaining: 695ms
73:	learn: 1036246.5747519	total: 670ms	remaining: 688ms
74:	learn: 1036246.6776069	total: 677ms	remaining: 677ms
75:	learn: 1034747.4200253	total: 687ms	remaining: 669ms
76:	learn: 1034747.5504028	total: 695ms	remaining: 659ms
77:	learn: 1034753.1391392	total: 703ms	remaining: 649ms
78:	learn: 1034753.2473644	total: 711ms	remaining: 639ms
79:	learn: 1034685.9360114	total: 719ms	remaining: 629ms
80:	learn: 1034167.9557981	total: 731ms	remaining: 622ms
81:	learn: 1034168.0481552	total: 738ms	remaining: 612ms
82:	learn: 1033577.4104180	total: 745ms	remaining: 602ms
83:	learn: 1033577.4151852	total: 752ms	remaining: 591ms
84:	learn: 1031865.4463003	total: 764ms	remaining: 584ms
85:	learn: 1027112.6062747	tota

[Parallel(n_jobs=1)]: Done 150 out of 150 | elapsed: 30.5min finished


0:	learn: 1418739.3242277	total: 183ms	remaining: 27.3s
1:	learn: 1265922.6923088	total: 358ms	remaining: 26.5s
2:	learn: 1134341.9212996	total: 544ms	remaining: 26.7s
3:	learn: 1028878.2034165	total: 713ms	remaining: 26s
4:	learn: 940973.4596771	total: 904ms	remaining: 26.2s
5:	learn: 871039.5850499	total: 1.14s	remaining: 27.4s
6:	learn: 810752.0143978	total: 1.46s	remaining: 29.8s
7:	learn: 762116.2052144	total: 1.7s	remaining: 30.2s
8:	learn: 723502.3414064	total: 1.9s	remaining: 29.8s
9:	learn: 691008.4419191	total: 2.1s	remaining: 29.4s
10:	learn: 664685.5104782	total: 2.29s	remaining: 28.9s
11:	learn: 642969.2474602	total: 2.54s	remaining: 29.2s
12:	learn: 625034.1497664	total: 2.87s	remaining: 30.2s
13:	learn: 609571.0226388	total: 3.14s	remaining: 30.5s
14:	learn: 596196.0865991	total: 3.33s	remaining: 30s
15:	learn: 585280.9211069	total: 3.53s	remaining: 29.6s
16:	learn: 576542.8623136	total: 3.7s	remaining: 29s
17:	learn: 568421.3268133	total: 3.9s	remaining: 28.6s
18:	learn

147:	learn: 413732.7561912	total: 34.4s	remaining: 465ms
148:	learn: 413303.8027262	total: 34.6s	remaining: 232ms
149:	learn: 413006.1197979	total: 34.7s	remaining: 0us


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
                   estimator=<catboost.core.CatBoostRegressor object at 0x7feac4c507b8>,
                   iid='warn', n_iter=50, n_jobs=None,
                   param_distributions={'border_count': [64, 128, 150, 176,
                                                         254],
                                        'colsample_bylevel': [0.01, 0.1, 0.3,
                                                              0.5, 0.7, 1],
                                        'depth': [6, 8, 10, 12],
                                        'l2_leaf_reg': [0, 1, 2, 3, 5, 8, 10,
                                                        15, 30, 50],
                                        'learning_rate': [0.05, 0.1, 0.15, 0.2],
                                        'od_type': ['Iter'],
                                        'od_wait': [10, 15, 30, 50, 80, 100]},
                   pre_dispatch='2*n_jobs', random_state=None, refit=Tru

In [18]:
display(rs_cv.best_params_)
from sklearn.externals import joblib
# Guarda el modelo
joblib.dump(rs_cv, 'catboost_rscv_4.pkl', compress=1)

{'od_wait': 50,
 'od_type': 'Iter',
 'learning_rate': 0.15,
 'l2_leaf_reg': 3,
 'depth': 12,
 'colsample_bylevel': 0.5,
 'border_count': 128}

['catboost_rscv_4.pkl']

In [19]:
results = pd.DataFrame(rs_cv.cv_results_).sort_values('rank_test_score')

In [14]:
results[['param_learning_rate',
'param_depth',
'param_od_type',
'param_od_wait',
'param_l2_leaf_reg', 'param_border_count', 'mean_test_score', 'rank_test_score']].head(50)

,param_learning_rate,param_depth,param_od_type,param_od_wait,param_l2_leaf_reg,param_border_count,mean_test_score,rank_test_score
41,0.2,12,Iter,15,5,128,-4.928761e+05,1
18,0.2,12,Iter,10,1,254,-4.941032e+05,2
34,0.2,10,Iter,50,3,150,-4.942549e+05,3
11,0.1,12,Iter,15,5,254,-4.970346e+05,4
8,0.2,10,Iter,30,15,254,-4.977549e+05,5
2,0.15,12,Iter,100,15,176,-4.981764e+05,6
12,0.15,12,Iter,100,8,254,-5.002439e+05,7
3,0.1,10,Iter,50,1,150,-5.007248e+05,8
1,0.2,10,Iter,80,3,150,-5.013951e+05,9
42,0.1,10,Iter,80,5,254,-5.037362e+05,10


### hyperopt

In [ ]:

features =['habitaciones', 'banos','metroscubiertos', 'metrostotales', 'count_id_zona', 'antiguedad',
           'garages', 'gimnasio', 'usosmultiples', 'piscina', 'gimnasio',
           'promedio_id_zona', 'promedio_por_mes', 'promedio_precio_hbg_tipo_propiedad_provincia',
            'tam_ambientes', 'promedio_precio_tipo_propiedad_ciudad', 'distancia_centro_mexico',
            'distancia_ciudad_centrica', 'distancia_ciudad_cara', 'varianza_precio_ciudad', 
            'promedio_precio_habitaciones_banos_garages','promedio_precio_tipo_propiedad_ciudad']



x_train, x_test, y_train, y_test = utils.dividir_dataset(df_train_f, 'precio', features, test_size=0.15)


D_train = catboost.Pool(x_train, y_train)
D_test = catboost.Pool(x_test, y_test)

def get_catboost_params(space):
    params = dict()
    params['learning_rate'] = space['learning_rate']
    params['depth'] = int(space['depth'])
    params['l2_leaf_reg'] = space['l2_leaf_reg']
    return params


obj_call_count = 0
cur_best_loss = np.inf
def objective(space):
    
    params = get_catboost_params(space)

    model = catboost.CatBoostClassifier(iterations=200,
                                        learning_rate=params['learning_rate'],
                                        depth=int(params['depth']),
                                        eval_metric='MAE',
                                        l2_leaf_reg=params['l2_leaf_reg'],
                                        early_stopping_rounds=3000,
                                        od_type="Iter",
                                        verbose= False
                                        )
    
    model.fit(D_train, eval_set=D_test, verbose= False)
    
    y_pred = model.predict_proba(D_test.get_features())
    
    return utils.MAE(df_test_f['precio'].values, y_pred)

In [ ]:
#  space = {
#          'depth': hp.quniform("depth", 1, 6, 10),
#          'l2_leaf_reg': hyperopt.hp.qloguniform('l2_leaf_reg', 0, 2, 1),
#         'learning_rate': hyperopt.hp.uniform('learning_rate', 1e-3, 5e-1),
#         }


# best = hyperopt.fmin(fn=objective,
#                      space=space,
#                      algo=tpe.suggest,
#                      max_evals=200,
#                     verbose=1)

# print('-'*50)
# print('The best params:')
# print( best )

In [11]:
# features = ['habitaciones', 'garages','banos','antiguedad', 'metroscubiertos',  'metrostotales',  'provincia', 'tipodepropiedad',
#             'gimnasio', 'usosmultiples', 'piscina', 'fecha']

features = ['garages', 'banos', 'metroscubiertos', 'metrostotales',
       'lat', 'lng', 'fecha', 'porcentaje_metros', 'diferencia_metros',
       'promedio_metros_tipo_propiedad', 'promedio_metros_cubiertos_provincia', 'mes', 
       'dias_desde_datos', 'tam_ambientes',
       'promedio_precio_provincia', 'promedio_precio_ciudad',
       'count_ciudad', 'promedio_id_zona',
       'count_id_zona', 'promedio_precio_tipo_propiedad',
       'promedio_precio_tipo_propiedad_ciudad',
       'count_tipo_propiedad_ciudad', 'promedio_por_mes', 'promedio_precio_hbg_tipo_propiedad',
       'puntaje', 'distancia_ciudad_centrica',
       'distancia_centro_mexico', 'provincia', 'ciudad', 'tipodepropiedad',
       'ranking_en_provincia_tipodepropiedad_precio',
       'ranking_en_provincia_tipodepropiedad_cantidad'] + ['provincia', 'tipodepropiedad', 'ciudad']

x_train, x_test, y_train, y_test = utils.dividir_dataset(df_train_f, 'precio', features, test_size=0.15)


# hopts = {'border_count': 176,
#  'depth': 10,
#  'l2_leaf_reg': 0,
#  'learning_rate': 0.2,
#  'od_type': 'Iter',
#  'od_wait': 10}

hopts = {'od_wait': 50,
    'od_type': 'Iter',
    'learning_rate': 0.15,
    'l2_leaf_reg': 3,
    'depth': 12,
    'colsample_bylevel': 0.5,
    'border_count': 128,
    'silent': True,
    }

# train_labels = 'precio'
# Initialize CatBoostRegressor
model = CatBoostRegressor(iterations=300,
                             eval_metric='MAE', **hopts)
# Fit model
model.fit(x_train, y_train, cat_features=['provincia', 'ciudad', 'tipodepropiedad'], eval_set=(x_test, y_test),plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [10]:
y_pred_test = model.predict(x_test)
y_pred_train = model.predict(x_train)
y_pred_eval = model.predict(utils.filtrar_features(df_test_f, features))

cat_mae_train = utils.MAE(y_train, y_pred_train)
cat_mae_test = utils.MAE(y_test, y_pred_test)
cat_mae_eval = utils.MAE(df_test_f['precio'], y_pred_eval)

print(f"MAE LightGBM (train): {cat_mae_train:.5f}")
print(f"MAE LightGBM (test): {cat_mae_test:.5f}")
print(f"MAE LightGBM (eval): {cat_mae_eval:.5f}")

MAE LightGBM (train): 385460.35320
MAE LightGBM (test): 480354.75748
MAE LightGBM (eval): 531375.79359
